In [88]:
import sys
import os
work_path = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
sys.path.append(work_path)

from helper.regrassion import *
from helper.util import *
from helper.plot import *
from helper.analysis import *


### [2] 데이터 가져오기

중고차 가격 예측을 위한 데이터 세트

https://www.kaggle.com/datasets/rishabhkarn/used-car-dataset

| 변수명 | 설명 |
|:--:|--:|
| car_name | Name of the car |
| registration_year | Vehicle registration year |
| insurance_validity | Type of insurance |
| fuel_type | Type of fuel |
| seats | Number of seats in the car |
| kms_driven | Total kms driven |
| ownsership | Number of ownership |
| transmission | Transmission mode |
| manufacturing_year | manufacturing year |
| mileage(kmpl) | Mileage of the car |
| engine(cc) | Engine capacity |
| max_power(bhp) | max power |
| torque(Nm) | torque |
| price(in lakhs) | Price of the used car |

In [89]:
origin = my_read_data("./res/Used_Car_Dataset.csv",info= False).drop('Unnamed: 0',axis=1)
df = origin.copy()
del_num=[457,633,653,1299,935,916,463,1463,388,554,574]
del_num.sort(reverse=True)
for i in del_num:
    df = df.drop(i,axis=0)
df=df.drop_duplicates()

In [90]:
brand = []
for i in df['car_name']:
    brand.append(i.split(' ')[1])
df['brand'] = brand
df['brand'].unique()

array(['Mercedes-Benz', 'Nissan', 'BMW', 'Kia', 'Skoda', 'Honda',
       'Hyundai', 'Tata', 'Renault', 'Ford', 'Jeep', 'MG', 'Maruti',
       'Audi', 'Toyota', 'Jaguar', 'Volkswagen', 'Mahindra', 'Volvo',
       'Land', 'Isuzu', 'Mitsubishi', 'Datsun', 'Lexus', 'Mini', 'Fiat',
       'Lamborghini'], dtype=object)

In [91]:
A = []
A_ = []
for i in origin['registration_year'].unique():
    try : 
        int(i)
        A.append(i)
    except:A_.append(i)
# A, A_

In [92]:
origin['registration_year'].unique()

map_registration_year = {a:None for a in A}
for i in A_:
    map_registration_year[i] = i
map_registration_year['Third Party insurance'] = None
df['registration_year'] = df['registration_year'].map(map_registration_year)
# df['registration_year']

In [93]:
month_dict = {'Apr':4,
 'Aug':8,
 'Feb':2,
 'Dec':12,
 'Jan':1,
 'Jul':7,
 'Jun':6,
 'Mar':3,
 'May':5,
 'Nov':11,
 'Oct':10,
 'Sep':9}

In [94]:
df['registration_year']

0       Jul-17
1       Jan-21
2       Sep-18
3       Dec-19
4       Aug-19
         ...  
1548    Aug-20
1549      None
1550    Jun-17
1551    May-18
1552    May-18
Name: registration_year, Length: 1125, dtype: object

In [95]:
month = []
year = []

for i in df['registration_year']:
    if i:
        try:
            month.append(month_dict[i[:3]])
            year.append(int(i[-2:]))
        except : print(i)
    else:
        month.append(None)
        year.append(None)

In [96]:
df['month']=month
df['year']=year

In [97]:
origin['insurance_validity'].value_counts()
map_insurance_validity = {a:a for a in origin['insurance_validity'].unique()}
map_insurance_validity['Petrol'] = None
map_insurance_validity['Not Available'] = None
map_insurance_validity['Third Party'] = 'Third Party insurance'
df['insurance_validity'] = df['insurance_validity'].map(map_insurance_validity)
df['insurance_validity']

0               Comprehensive
1               Comprehensive
2               Comprehensive
3               Comprehensive
4               Comprehensive
                ...          
1548            Comprehensive
1549    Third Party insurance
1550            Comprehensive
1551            Comprehensive
1552            Comprehensive
Name: insurance_validity, Length: 1125, dtype: object

In [98]:
origin['fuel_type'].value_counts()
map_fuel_type = {a:a for a in origin['fuel_type'].unique()}
map_fuel_type['CNG'] = None
map_fuel_type['5 Seats'] = None
df['fuel_type'] = df['fuel_type'].map(map_fuel_type)
df['fuel_type']

0       Petrol
1       Petrol
2       Diesel
3       Petrol
4       Petrol
         ...  
1548    Diesel
1549    Petrol
1550    Petrol
1551    Petrol
1552    Petrol
Name: fuel_type, Length: 1125, dtype: object

In [99]:
origin['seats'].value_counts()
map_seats = {a:a if a<10 else None for a in origin['seats'].unique()}
df['seats'] = df['seats'].map(map_seats)
df['seats']

0       5.0
1       5.0
2       5.0
3       5.0
4       5.0
       ... 
1548    5.0
1549    5.0
1550    5.0
1551    5.0
1552    5.0
Name: seats, Length: 1125, dtype: float64

In [100]:
origin['ownsership'].value_counts()
map_ownsership = {a:a if "Owner" in a else None for a in origin['ownsership'].unique()}
df['ownsership'] = df['ownsership'].map(map_ownsership)
df['ownsership']


0        First Owner
1        First Owner
2        First Owner
3        First Owner
4        First Owner
            ...     
1548     First Owner
1549            None
1550     First Owner
1551    Second Owner
1552     First Owner
Name: ownsership, Length: 1125, dtype: object

In [101]:
A = []
A_ = []
for i in origin['transmission'].unique():
    try : 
        int(i)
        A.append(i)
    except:A_.append(i)
origin['transmission'].value_counts()

Manual                 835
Automatic              668
2017                    28
2014                     5
2011                     3
2023                     2
2020                     2
2021                     2
2022                     2
Power Windows Front      2
2018                     2
2015                     1
2016                     1
Name: transmission, dtype: int64

In [102]:
map_transmission = {a:None for a in A}
for i in A_:
    map_transmission[i] = i
map_transmission['Power Windows Front']=None
df['transmission'] = df['transmission'].map(map_transmission)
df['transmission']

0       Automatic
1       Automatic
2       Automatic
3          Manual
4       Automatic
          ...    
1548    Automatic
1549         None
1550       Manual
1551       Manual
1552       Manual
Name: transmission, Length: 1125, dtype: object

In [103]:
origin['manufacturing_year'].value_counts()
A = []
A_ = []
for i in origin['manufacturing_year'].unique():
    try : 
        int(i)
        A.append(i)
    except:A_.append(i)
A, A_

(['2017',
  '2020',
  '2018',
  '2019',
  '2015',
  '2021',
  '2014',
  '2022',
  '2010',
  '2011',
  '2023',
  '2016',
  '2009',
  '2013',
  '2012',
  '2007'],
 ['Power Steering', 'Power Windows Front', 'Air Conditioner'])

In [104]:
map_manufacturing_year = {a:a for a in A}
for i in A_:
    map_manufacturing_year[i] = None
df['manufacturing_year'] = df['manufacturing_year'].map(map_manufacturing_year)
df['manufacturing_year']

0       2017
1       2020
2       2018
3       2019
4       2019
        ... 
1548    2020
1549    None
1550    2017
1551    2018
1552    2018
Name: manufacturing_year, Length: 1125, dtype: object

In [105]:
for i,v in enumerate(origin['mileage(kmpl)']):
    if v>40 : df['mileage(kmpl)'][i] = None


In [106]:
max = 13519
min = 352

for i,v in enumerate(origin['engine(cc)']):
    if v < min or v > max : df['engine(cc)'][i] = None


In [107]:
df.dtypes

car_name               object
registration_year      object
insurance_validity     object
fuel_type              object
seats                 float64
kms_driven              int64
ownsership             object
transmission           object
manufacturing_year     object
mileage(kmpl)         float64
engine(cc)            float64
max_power(bhp)        float64
torque(Nm)            float64
price(in lakhs)       float64
brand                  object
month                 float64
year                  float64
dtype: object

In [108]:
df2 = my_set_category(df.dropna().drop('car_name',axis=1))
df2['price'] = df2['price(in lakhs)']
df2.drop('price(in lakhs)',axis=1,inplace=True)

df2.dtypes

registration_year     category
insurance_validity    category
fuel_type             category
seats                  float64
kms_driven               int64
ownsership            category
transmission          category
manufacturing_year    category
mileage(kmpl)          float64
engine(cc)             float64
max_power(bhp)         float64
torque(Nm)             float64
brand                 category
month                  float64
year                   float64
price                  float64
dtype: object

In [109]:
df2['price'].describe()


count      902.000000
mean       119.401086
std       3162.729844
min          1.100000
25%          5.010000
50%          6.950000
75%         14.950000
max      95000.000000
Name: price, dtype: float64

In [110]:
import numpy as np
a = (int(df2['year'].max())-np.array(df2['year']) ) *12 + df2['month']
df2['registration'] = a
df2['mileage'] = df2['mileage(kmpl)']
df2['engine'] = df2['engine(cc)']
df2['max_power'] = df2['max_power(bhp)']
df2['torque'] = df2['torque(Nm)']
df2.drop(['torque(Nm)','engine(cc)','max_power(bhp)','mileage(kmpl)'],axis=1,inplace=True)
df2['kms_driven2']=df2['kms_driven']**2
df2['registration2'] = df2['registration']**2
df2['torque2'] = df2['torque']**2
df2['max_power2'] = df2['max_power']**2
df2['engine2'] = df2['engine']**2
df2['mileage2'] = df2['mileage'] **2
df2['seats2'] = df2['seats']**2

In [111]:

df3 =df2.drop(['year','month','manufacturing_year','registration_year'],axis=1).reset_index(drop=True).drop(683)
df3.head()


,insurance_validity,fuel_type,seats,kms_driven,ownsership,transmission,brand,price,registration,mileage,engine,max_power,torque,kms_driven2,registration2,torque2,max_power2,engine2,mileage2,seats2
0,Comprehensive,Petrol,5.0,56000,First Owner,Automatic,Mercedes-Benz,63.75,79.0,7.81,2996.0,2996.0,333.0,3136000000,6241.0,110889.0,8976016.0,8976016.0,60.9961,25.0
1,Comprehensive,Petrol,5.0,30615,First Owner,Automatic,Nissan,8.99,25.0,17.40,999.0,999.0,9863.0,937278225,625.0,97278769.0,998001.0,998001.0,302.7600,25.0
2,Comprehensive,Diesel,5.0,24000,First Owner,Automatic,BMW,23.75,69.0,20.68,1995.0,1995.0,188.0,576000000,4761.0,35344.0,3980025.0,3980025.0,427.6624,25.0
3,Comprehensive,Petrol,5.0,18378,First Owner,Manual,Kia,13.56,60.0,16.50,1353.0,1353.0,13808.0,337750884,3600.0,190660864.0,1830609.0,1830609.0,272.2500,25.0
4,Comprehensive,Petrol,5.0,44900,First Owner,Automatic,Skoda,24.00,56.0,14.67,1798.0,1798.0,17746.0,2016010000,3136.0,314920516.0,3232804.0,3232804.0,215.2089,25.0


In [112]:
categories = [x for x in df3.columns if df3[x].dtypes == 'category']


In [113]:
for c in categories:
    tmp = df3[[c, 'price']]
    my_anova(tmp, hue=c, target='price')

pingouin.anova
+----+--------------------+---------+------+---------+---------+----------+------------+
|    | Source             |      SS |   DF |      MS |       F |    p-unc |        np2 |
|----+--------------------+---------+------+---------+---------+----------+------------|
|  0 | insurance_validity | 2534.78 |    2 | 1267.39 | 4.46579 | 0.011752 | 0.00984814 |
|  1 | Within             |  254852 |  898 | 283.799 |     nan |      nan |        nan |
+----+--------------------+---------+------+---------+---------+----------+------------+

statsmodels.anova.anova_lm
+-----------------------+------+----------+-----------+---------+----------+
|                       |   df |   sum_sq |   mean_sq |       F |   PR(>F) |
|-----------------------+------+----------+-----------+---------+----------|
| C(insurance_validity) |    2 |  2534.78 |   1267.39 | 4.46579 | 0.011752 |
| Residual              |  898 |   254852 |   283.799 |     nan |      nan |
+-----------------------+------+------

In [114]:
df_brand = df3[['brand','price']]
df_brand_name = df_brand.groupby('brand').mean().sort_values('price').reset_index()['brand']
map_brand = {}
for i,v in enumerate(df_brand_name):
    map_brand[v] = i//5
df3['brand_value'] = df3['brand'].map(map_brand)
df3.drop('brand',axis=1,inplace=True)

In [116]:
dummy_df = my_dummies(std_df)
my_pretty_table(dummy_df.head())


+----+-----------+--------------+----------------+-----------+-----------+-------------+------------+---------------+-----------------+------------+--------------+-----------+------------+-----------+---------------+---------+--------------------------------------------+-------------------------------+--------------------+-----------------------+
|    |     seats |   kms_driven |   registration |   mileage |    engine |   max_power |     torque |   kms_driven2 |   registration2 |    torque2 |   max_power2 |   engine2 |   mileage2 |    seats2 |   brand_value |   price |   insurance_validity_Third Party insurance |   insurance_validity_Zero Dep |   fuel_type_Petrol |   transmission_Manual |
|----+-----------+--------------+----------------+-----------+-----------+-------------+------------+---------------+-----------------+------------+--------------+-----------+------------+-----------+---------------+---------+--------------------------------------------+-------------------------------

In [132]:
x_train, x_test, y_train, y_test = my_train_test_split(dummy_df, "price", test_size=0.3)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((630, 19), (271, 19), (630,), (271,))

In [134]:
ignore = []
for i in x_train.columns:
    check = False
    for j in x_train[i]:
        check = check or j
        if check:
            break
    else:
        ignore.append(i)
    check = False
    for j in x_test[i]:
        check = check or j
        if check:
            break 
    else:
        ignore.append(i)
ignore


[]

In [140]:
ignore = ignore + ['engine','max_power2','seats2','mileage','torque2','max_power','registration','insurance_validity_Zero Dep','kms_driven2','insurance_validity_Third Party insurance','torque','kms_driven','mileage2','seats']
x_train2 = x_train.drop(ignore,axis=1)
x_test2 = x_test.drop(ignore,axis=1)
my_linear_regrassion(x_train2, y_train, x_test2, y_test,use_plot=False,resid_test=False)
# 0행렬이 포함되어있는경우 `LinAlgError: Singular matrix` 발생, 해결방법은 해당 행렬을 찾아서 삭제한다.

[회귀식]
price = -2.949 * registration2 + 4.986 * engine2 + 8.641 * brand_value + 2.695 * fuel_type_Petrol + -3.274 * transmission_Manual + 14.171

[회귀분석 성능평가]
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+
|        |   결정계수(R2) |   평균절대오차(MAE) |   평균제곱오차(MSE) |   평균오차(RMSE) |   평균 절대 백분오차 비율(MAPE) |   평균 비율 오차(MPE) |
|--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------|
| 데이터 |       0.656008 |             6.01176 |             90.8615 |          9.53213 |                         81.8134 |              -18.3053 |
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+

[선형회귀분석 결과보고]
+----+------------+---------------------+--------------------+------------+------------------+--------+------------+------

LinearRegression(n_jobs=-1)